In [2]:
pip install llama-cpp-haystack

  Using cached llama_cpp_haystack-1.0.0-py3-none-any.whl.metadata (10 kB)
  Using cached haystack_ai-2.12.1-py3-none-any.whl.metadata (14 kB)
  Using cached llama_cpp_python-0.3.8.tar.gz (67.3 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached haystack_experimental-0.8.0-py3-none-any.whl.metadata (12 kB)
  Using cached lazy_imports-0.4.0-py3-none-any.whl.metadata (10 kB)
  Using cached posthog-3.24.1-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached diskcache-5.6.3-py3-none-any.whl.metadata (20 kB)
  Using cached monotonic-1.6-py2.py3-none-any.whl.metadata (1.5 kB)
  Using cached backoff-2.2.1-py3-none-any.whl.metadata (14 kB)
Using cached llama_cpp_haystack-1.0.0-py3-none-any.whl (13 kB)
Using cached haystack_ai-2.12.1-py3-none-any.whl (482 kB)
Using cached diskcache-5.6.3-py3-none-any.whl (45 kB)
Using cached posthog-3.24.1-py2.py3-none

In [3]:
from haystack import Document, Pipeline
from haystack.components.builders.answer_builder import AnswerBuilder
from haystack.components.builders.prompt_builder import PromptBuilder
from haystack.components.embedders import SentenceTransformersDocumentEmbedder, SentenceTransformersTextEmbedder
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever
from haystack.components.writers import DocumentWriter
from haystack.document_stores.in_memory import InMemoryDocumentStore

# Import LlamaCppGenerator
from haystack_integrations.components.generators.llama_cpp import LlamaCppGenerator

In [4]:
import pandas as pd

In [5]:
from google.colab import drive
drive.mount('/content/drive')
file_path = '/content/drive/My Drive/df_for_LLM_v2.csv'
df_for_LLM = pd.read_csv(file_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
pd.set_option('display.max_colwidth', None)

In [7]:
df_for_LLM

,name,property_tag_old_2,property_email,property_hours_of_operation,property_location,property_description,property_phone_number,combined_text
0,San Carlos,Hardware Store,mmora@depositosancarlos.com,"Monday - Friday 7:00 AM- 5:00 PM, Saturday 7:00 AM- 3:00 PM, Sunday Сlosed",Playa del Coco,"Deposito San Carlos is a largest building materials store with a strong presence in Guanacaste since 1994. Starting as a small wood depot, it has expanded to multiple branches, providing a wide range of construction materials and expert advice.\n\n• Services: Paint consultations, material sales, delivery fleet\n• Experienced staff",+506 2670-1211,"San Carlos, Hardware Store, mmora@depositosancarlos.com, Monday - Friday 7:00 AM- 5:00 PM, Saturday 7:00 AM- 3:00 PM, Sunday Сlosed, Playa del Coco, Deposito San Carlos is a largest building materials store with a strong presence in Guanacaste since 1994. Starting as a small wood depot, it has expanded to multiple branches, providing a wide range of construction materials and expert advice.\n\n• Services: Paint consultations, material sales, delivery fleet\n• Experienced staff, +506 2670-1211"
1,Angel's salon,Beauty Bar,myrnaangel@hotmail.com,"Monday 9:00 AM - 6:00 PM, Tuesday - Saturday 9:00 AM - 5:00 PM, Sunday - Сlosed",Playa del Coco,"Angel's Salon is a professional hair and beauty salon with 19 years of experience. Offers a wide range of services for men, women, and children. \n\n• Hair coloring and extensions\n• Manicure and pedicure services\n• Eyelash extensions\n• Professional products",no information available,"Angel's salon, Beauty Bar, myrnaangel@hotmail.com, Monday 9:00 AM - 6:00 PM, Tuesday - Saturday 9:00 AM - 5:00 PM, Sunday - Сlosed, Playa del Coco, Angel's Salon is a professional hair and beauty salon with 19 years of experience. Offers a wide range of services for men, women, and children. \n\n• Hair coloring and extensions\n• Manicure and pedicure services\n• Eyelash extensions\n• Professional products, no information available"
2,La Casa del Plastico,Department Store,info@lacasadelplastico.com,"Monday - Saturday 8:00 AM - 6:00 PM, Sunday Closed",Playa del Coco,"La Casa del Plastico specializes in the sale of products for home, primary food industry, and commerce.",+506 2670-0700,"La Casa del Plastico, Department Store, info@lacasadelplastico.com, Monday - Saturday 8:00 AM - 6:00 PM, Sunday Closed, Playa del Coco, La Casa del Plastico specializes in the sale of products for home, primary food industry, and commerce., +506 2670-0700"
3,EPA Solarium,"Department Store, Hardware Store",contacto@xr.epa.biz,"Monday - Friday 9:00 AM - 5:00 PM, Saturday 9:00 AM - 3:00 PM, Sunday closed",Liberia,"EPA Solarium - Hardware and Home Goods Store offers a wide selection of building materials, furniture, and home improvement products. Еру store provides everything needed for construction, decoration, and renovation projects.\n\n• Building materials and finishes\n• Furniture and home decor\n• Lighting and garden supplies\n• Automotive and tools\n• Wood and paint\n• Appliances and bathroom fixtures",no information available,"EPA Solarium, Department Store, Hardware Store, contacto@xr.epa.biz, Monday - Friday 9:00 AM - 5:00 PM, Saturday 9:00 AM - 3:00 PM, Sunday closed, Liberia, EPA Solarium - Hardware and Home Goods Store offers a wide selection of building materials, furniture, and home improvement products. Еру store provides everything needed for construction, decoration, and renovation projects.\n\n• Building materials and finishes\n• Furniture and home decor\n• Lighting and garden supplies\n• Automotive and tools\n• Wood and paint\n• Appliances and bathroom fixtures, no information available"
4,Policia Upala,"Police Department, Information Services",no information available,"Monday - Friday 8:00 AM - 4:00 PM, Saturday - Sunday Closed",Canalete,Policia Upala - Police Department and Information Services provides essential public safety services and handles regional documentation processes. Fi

In [ ]:
# # Пример №1, в контенте только описание заведений
# docs = [
#     Document(
#         content=f"{row['property_description']}",
#         meta={
#             "name": row['name'],
#             "tags": row['property_tag_old_2'],
#             "email": row['property_email'],
#             "location": row['property_location'],
#             "phone_number": row['property_phone_number'],
#             "opening_hours": row['property_hours_of_operation']
#         }
#     )
#     for _, row in df_for_LLM.iterrows()
# ]

In [8]:
# Пример №3, в контенте combined_text то есть содержимое всех столбцов
docs = [
    Document(
        content=f"{row['combined_text']}",

    )
    for _, row in df_for_LLM.iterrows()
]

In [9]:
doc_store = InMemoryDocumentStore(embedding_similarity_function="cosine")
doc_embedder = SentenceTransformersDocumentEmbedder(model="sentence-transformers/all-MiniLM-L6-v2")

# Indexing Pipeline
indexing_pipeline = Pipeline()
indexing_pipeline.add_component(instance=doc_embedder, name="DocEmbedder")
indexing_pipeline.add_component(instance=DocumentWriter(document_store=doc_store), name="DocWriter")
indexing_pipeline.connect("DocEmbedder", "DocWriter")
#indexing_pipeline.connect(connect_from="DocEmbedder", connect_to="DocWriter")                         не работает в новейшей версии

indexing_pipeline.run({"DocEmbedder": {"documents": docs}})

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/5 [00:00<?, ?it/s]

{'DocWriter': {'documents_written': 148}}

In [10]:
prompt_template = """GPT4 Correct User: Answer the question using the provided context.
Question: {{question}}
Context:
The following table provides information about restaurants:

{{ df_text }}

Use the information from the table to answer the question.
<|end_of_turn|>
GPT4 Correct Assistant:
"""

In [ ]:
# Prompt Template for the https://huggingface.co/openchat/openchat-3.5-1210 LLM
prompt_template = """GPT4 Correct User: Answer the question using the provided context.
Question: {{question}}
Context:
{% for doc in documents %}
    {{ doc.content }}
    Use the information from the table to answer the question.
{% endfor %}
<|end_of_turn|>
GPT4 Correct Assistant:
"""

rag_pipeline = Pipeline()

text_embedder = SentenceTransformersTextEmbedder(model="sentence-transformers/all-MiniLM-L6-v2")

# Load the LLM using LlamaCppGenerator
model_path = "/content/drive/MyDrive/openchat-3.5-1210.Q3_K_S.gguf"
generator = LlamaCppGenerator(model=model_path, n_ctx=4096, n_batch=128)

rag_pipeline.add_component(
    instance=text_embedder,
    name="text_embedder",
)
rag_pipeline.add_component(instance=InMemoryEmbeddingRetriever(document_store=doc_store, top_k=3), name="retriever")
rag_pipeline.add_component(instance=PromptBuilder(template=prompt_template), name="prompt_builder")
rag_pipeline.add_component(instance=generator, name="llm")
rag_pipeline.add_component(instance=AnswerBuilder(), name="answer_builder")

rag_pipeline.connect("text_embedder", "retriever")
rag_pipeline.connect("retriever", "prompt_builder.documents")
rag_pipeline.connect("prompt_builder", "llm")
rag_pipeline.connect("llm.replies", "answer_builder.replies")
rag_pipeline.connect("retriever", "answer_builder.documents")

🚅 Components
  - text_embedder: SentenceTransformersTextEmbedder
  - retriever: InMemoryEmbeddingRetriever
  - prompt_builder: PromptBuilder
  - llm: LlamaCppGenerator
  - answer_builder: AnswerBuilder
🛤️ Connections
  - text_embedder.embedding -> retriever.query_embedding (List[float])
  - retriever.documents -> prompt_builder.documents (List[Document])
  - retriever.documents -> answer_builder.documents (List[Document])
  - prompt_builder.prompt -> llm.prompt (str)
  - llm.replies -> answer_builder.replies (List[str])

In [ ]:
question = "When is open the restaurant Zi Lounge?"
result = rag_pipeline.run(
    {
        "text_embedder": {"text": question},
        "prompt_builder": {"question": question},
        "llm": {"generation_kwargs": {"max_tokens": 128, "temperature": 0.1}},
        "answer_builder": {"query": question},
    }
)

generated_answer = result["answer_builder"]["answers"][0]
print(generated_answer.data)


llama_model_loader: loaded meta data with 23 key-value pairs and 291 tensors from /content/drive/MyDrive/openchat-3.5-1210.Q3_K_S.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = openchat_openchat-3.5-1210
llama_model_loader: - kv   2:                       llama.context_length u32              = 8192
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:           

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

llama_perf_context_print:        load time =  124689.23 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   239 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    13 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =  134388.37 ms /   252 tokens


 The restaurant Zi Lounge is open at any convenient time.


При использовании cpu, время выполнения чут больше двух минут. Причем подавать нужно также combined_text похоже, так как при простом вопросе не учитывалась meta с режимом работы, а ответ извлекся тупо из description.

In [ ]:
question = "What are the opening hours of the restaurant Zi Lounge?"
result = rag_pipeline.run(
    {
        "text_embedder": {"text": question},
        "prompt_builder": {"question": question},
        "llm": {"generation_kwargs": {"max_tokens": 128, "temperature": 0.1}},
        "answer_builder": {"query": question},
    }
)

generated_answer = result["answer_builder"]["answers"][0]
print(generated_answer.data)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Llama.generate: 19 prefix-match hit, remaining 223 prompt tokens to eval
llama_perf_context_print:        load time =  124689.23 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   223 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    19 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =  131211.11 ms /   242 tokens


 The context does not provide specific information about the opening hours of the restaurant Zi Lounge.


При использовании cpu, время выполнения около трех минут при первом запуске (формирование батчей). При втором чуть больше 2х минут. Причем подавать нужно также combined_text похоже, так как при простом вопросе не учитывалась meta с режимом работы, а ответ извлекся тупо из description. Очень интересный ответ на второй вопрос.

Попробуем подать combined_text, задать те же вопросы, а также подключить gpu.

In [ ]:
question = "When is open the restaurant Zi Lounge?"
result = rag_pipeline.run(
    {
        "text_embedder": {"text": question},
        "prompt_builder": {"question": question},
        "llm": {"generation_kwargs": {"max_tokens": 128, "temperature": 0.1}},
        "answer_builder": {"query": question},
    }
)

generated_answer = result["answer_builder"]["answers"][0]
print(generated_answer.data)

llama_model_loader: loaded meta data with 23 key-value pairs and 291 tensors from /content/drive/MyDrive/openchat-3.5-1210.Q3_K_S.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = openchat_openchat-3.5-1210
llama_model_loader: - kv   2:                       llama.context_length u32              = 8192
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:           

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

llama_perf_context_print:        load time =  257022.70 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   480 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    25 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =  275015.25 ms /   505 tokens


 Zi Lounge is open Monday - Sunday from 11:00 AM to 2:30 AM.


In [ ]:
question = "What are the opening hours of the restaurant Zi Lounge?"
result = rag_pipeline.run(
    {
        "text_embedder": {"text": question},
        "prompt_builder": {"question": question},
        "llm": {"generation_kwargs": {"max_tokens": 128, "temperature": 0.1}},
        "answer_builder": {"query": question},
    }
)

generated_answer = result["answer_builder"]["answers"][0]
print(generated_answer.data)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Llama.generate: 19 prefix-match hit, remaining 464 prompt tokens to eval
llama_perf_context_print:        load time =  257022.70 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   464 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    29 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =  274426.72 ms /   493 tokens


 The opening hours of the restaurant Zi Lounge are Monday - Sunday 11:00 AM - 2:30 AM.


Время выполнения при первом запуске чуть больше 5 минут. При втором 4 минуты. Не уверен, что все работает через GPU. Ответ на вопрос при использовании combined_text полностью устраивает. При переключении на CPU, время работы тоже. Нужно как-то специально включить GPU видимо.

Попробуем вопрос из QA датасета. Целиком проверять алгоритм не хочу.

In [ ]:
question = "What Italian restaurants are there in Playa Hermosa?"
result = rag_pipeline.run(
    {
        "text_embedder": {"text": question},
        "prompt_builder": {"question": question},
        "llm": {"generation_kwargs": {"max_tokens": 128, "temperature": 0.1}},
        "answer_builder": {"query": question},
    }
)

generated_answer = result["answer_builder"]["answers"][0]
print(generated_answer.data)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Llama.generate: 20 prefix-match hit, remaining 428 prompt tokens to eval
llama_perf_context_print:        load time =  238477.72 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   428 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    15 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =  213556.77 ms /   443 tokens


 There is no information available about any Italian restaurants in Playa Hermosa.


Время работы 3,5 мин. Ответ неудовлетворительный.

In [ ]:
question = "When is open the restaurant Zi Lounge?"
result = rag_pipeline.run(
    {
        "text_embedder": {"text": question},
        "prompt_builder": {"question": question},
        "llm": {"generation_kwargs": {"max_tokens": 128, "temperature": 0.1}},
        "answer_builder": {"query": question},
    }
)

generated_answer = result["answer_builder"]["answers"][0]
print(generated_answer.data)

llama_model_loader: loaded meta data with 23 key-value pairs and 291 tensors from /content/drive/MyDrive/openchat-3.5-1210.Q3_K_S.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = openchat_openchat-3.5-1210
llama_model_loader: - kv   2:                       llama.context_length u32              = 8192
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:           

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

llama_perf_context_print:        load time =  299110.57 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   519 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    25 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =  317850.89 ms /   544 tokens


 Zi Lounge is open Monday - Sunday from 11:00 AM to 2:30 AM.


In [ ]:
question = "What Italian restaurants are there in Playa Hermosa?"
result = rag_pipeline.run(
    {
        "text_embedder": {"text": question},
        "prompt_builder": {"question": question},
        "llm": {"generation_kwargs": {"max_tokens": 128, "temperature": 0.1}},
        "answer_builder": {"query": question},
    }
)

generated_answer = result["answer_builder"]["answers"][0]
print(generated_answer.data)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Llama.generate: 19 prefix-match hit, remaining 468 prompt tokens to eval
llama_perf_context_print:        load time =  299110.57 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   468 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    14 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =  282279.40 ms /   482 tokens


 There is no information available about Italian restaurants in Playa Hermosa.


In [ ]:
# Prompt Template for the https://huggingface.co/openchat/openchat-3.5-1210 LLM
prompt_template = """GPT4 Correct User: Answer the question using the provided context.
Question: {{question}}
Context:
{% for doc in documents %}
    {{ doc.content }}
    Use the information from the table to answer the question.
{% endfor %}
<|end_of_turn|>
GPT4 Correct Assistant:
"""

rag_pipeline = Pipeline()

text_embedder = SentenceTransformersTextEmbedder(model="sentence-transformers/all-MiniLM-L6-v2")

# Load the LLM using LlamaCppGenerator
model_path = "/content/drive/MyDrive/openchat-3.5-1210.Q5_K_M.gguf"
generator = LlamaCppGenerator(model=model_path, n_ctx=4096, n_batch=128)

rag_pipeline.add_component(
    instance=text_embedder,
    name="text_embedder",
)
rag_pipeline.add_component(instance=InMemoryEmbeddingRetriever(document_store=doc_store, top_k=3), name="retriever")
rag_pipeline.add_component(instance=PromptBuilder(template=prompt_template), name="prompt_builder")
rag_pipeline.add_component(instance=generator, name="llm")
rag_pipeline.add_component(instance=AnswerBuilder(), name="answer_builder")

rag_pipeline.connect("text_embedder", "retriever")
rag_pipeline.connect("retriever", "prompt_builder.documents")
rag_pipeline.connect("prompt_builder", "llm")
rag_pipeline.connect("llm.replies", "answer_builder.replies")
rag_pipeline.connect("retriever", "answer_builder.documents")

🚅 Components
  - text_embedder: SentenceTransformersTextEmbedder
  - retriever: InMemoryEmbeddingRetriever
  - prompt_builder: PromptBuilder
  - llm: LlamaCppGenerator
  - answer_builder: AnswerBuilder
🛤️ Connections
  - text_embedder.embedding -> retriever.query_embedding (List[float])
  - retriever.documents -> prompt_builder.documents (List[Document])
  - retriever.documents -> answer_builder.documents (List[Document])
  - prompt_builder.prompt -> llm.prompt (str)
  - llm.replies -> answer_builder.replies (List[str])

In [ ]:
question = "What Italian restaurants are there in Playa Hermosa?"
result = rag_pipeline.run(
    {
        "text_embedder": {"text": question},
        "prompt_builder": {"question": question},
        "llm": {"generation_kwargs": {"max_tokens": 128, "temperature": 0.1}},
        "answer_builder": {"query": question},
    }
)

generated_answer = result["answer_builder"]["answers"][0]
print(generated_answer.data)

llama_model_loader: loaded meta data with 23 key-value pairs and 291 tensors from /content/drive/MyDrive/openchat-3.5-1210.Q5_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = openchat_openchat-3.5-1210
llama_model_loader: - kv   2:                       llama.context_length u32              = 8192
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:           

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

llama_perf_context_print:        load time =  299811.18 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   487 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    14 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =  312465.95 ms /   501 tokens


 There is no information available about Italian restaurants in Playa Hermosa.


In [ ]:
#from llama_cpp import Llama

In [11]:
# Prompt Template for the https://huggingface.co/openchat/openchat-3.5-1210 LLM
prompt_template = """GPT4 Correct User: Answer the question using the provided context.
Question: {{question}}
Context:
{% for doc in documents %}
    {{ doc.content }}
    Use the information from the table to answer the question.
{% endfor %}
<|end_of_turn|>
GPT4 Correct Assistant:
"""

rag_pipeline = Pipeline()

text_embedder = SentenceTransformersTextEmbedder(model="sentence-transformers/all-MiniLM-L6-v2")

# Load the LLM using LlamaCppGenerator

model_path = "/content/drive/MyDrive/Meta-Llama-3.1-8B-Instruct-Q6_K.gguf"
#model_path = "/content/drive/MyDrive/openchat-3.5-1210.Q5_K_M.gguf"
generator = LlamaCppGenerator(model=model_path, n_ctx=4096, n_batch=128)

rag_pipeline.add_component(
    instance=text_embedder,
    name="text_embedder",
)
rag_pipeline.add_component(instance=InMemoryEmbeddingRetriever(document_store=doc_store, top_k=3), name="retriever")
rag_pipeline.add_component(instance=PromptBuilder(template=prompt_template), name="prompt_builder")
rag_pipeline.add_component(instance=generator, name="llm")
rag_pipeline.add_component(instance=AnswerBuilder(), name="answer_builder")

rag_pipeline.connect("text_embedder", "retriever")
rag_pipeline.connect("retriever", "prompt_builder.documents")
rag_pipeline.connect("prompt_builder", "llm")
rag_pipeline.connect("llm.replies", "answer_builder.replies")
rag_pipeline.connect("retriever", "answer_builder.documents")

🚅 Components
  - text_embedder: SentenceTransformersTextEmbedder
  - retriever: InMemoryEmbeddingRetriever
  - prompt_builder: PromptBuilder
  - llm: LlamaCppGenerator
  - answer_builder: AnswerBuilder
🛤️ Connections
  - text_embedder.embedding -> retriever.query_embedding (List[float])
  - retriever.documents -> prompt_builder.documents (List[Document])
  - retriever.documents -> answer_builder.documents (List[Document])
  - prompt_builder.prompt -> llm.prompt (str)
  - llm.replies -> answer_builder.replies (List[str])

In [12]:
question = "What Italian restaurants are there in Playa Hermosa?"
result = rag_pipeline.run(
    {
        "text_embedder": {"text": question},
        "prompt_builder": {"question": question},
        "llm": {"generation_kwargs": {"max_tokens": 128, "temperature": 0.1}},
        "answer_builder": {"query": question},
    }
)

generated_answer = result["answer_builder"]["answers"][0]
print(generated_answer.data)

llama_model_loader: loaded meta data with 33 key-value pairs and 292 tensors from /content/drive/MyDrive/Meta-Llama-3.1-8B-Instruct-Q6_K.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Meta Llama 3.1 8B Instruct
llama_model_loader: - kv   3:                           general.finetune str              = Instruct
llama_model_loader: - kv   4:                           general.basename str              = Meta-Llama-3.1
llama_model_loader: - kv   5:                         general.size_label str              = 8B
llama_model_loader: - kv   6:                            general.license str              = llama3.1
llama_model_lo

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

llama_perf_context_print:        load time =  236411.57 ms
llama_perf_context_print: prompt eval time =  236411.18 ms /   396 tokens (  597.00 ms per token,     1.68 tokens per second)
llama_perf_context_print:        eval time =   96691.97 ms /    84 runs   ( 1151.09 ms per token,     0.87 tokens per second)
llama_perf_context_print:       total time =  333254.51 ms /   480 tokens


 Based on the provided context, there are two Italian restaurants mentioned in Playa Hermosa:

None. There are no Italian restaurants mentioned in the context. The restaurants mentioned are Roberto's Restaurant (International), Ginger (Asian-inspired tapas), and Santorini (Greek). If you would like to know more about Italian restaurants in Playa Hermosa, I can try to provide more information or help you find the answer.


In [13]:
question = "I need hardware store in playa del coco"
result = rag_pipeline.run(
    {
        "text_embedder": {"text": question},
        "prompt_builder": {"question": question},
        "llm": {"generation_kwargs": {"max_tokens": 128, "temperature": 0.1}},
        "answer_builder": {"query": question},
    }
)

generated_answer = result["answer_builder"]["answers"][0]
print(generated_answer.data)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Llama.generate: 17 prefix-match hit, remaining 296 prompt tokens to eval
llama_perf_context_print:        load time =  236411.57 ms
llama_perf_context_print: prompt eval time =  179007.26 ms /   296 tokens (  604.75 ms per token,     1.65 tokens per second)
llama_perf_context_print:        eval time =  131587.01 ms /   127 runs   ( 1036.12 ms per token,     0.97 tokens per second)
llama_perf_context_print:       total time =  310806.92 ms /   423 tokens


 Based on the provided context, I would recommend the following hardware store in Playa del Coco:

Unfortunately, there is no specific hardware store mentioned in the provided context. However, I can suggest a few options based on the information available:

* La Casa del Plastico: Although it specializes in products for home, primary food industry, and commerce, it might have some hardware-related products. You can contact them at +506 2670-0700 to inquire about their inventory.
* Super New China: This is a convenient store that offers a wide range of products, including household items. They might have some basic hardware items, but
